In [ ]:
import numpy as np
import pandas as pd
import xlwings as xw
import os
from scipy.stats import truncnorm
import time

timeinitial = time.time()
file_name = '.xlsx'
file_path = os.path.join(os.getcwd(), file_name)

# Parameters for the Monte Carlo simulation
mean_wire_harness_price = 0.87*2.20462  # Converted to USD/kg from USD/lb
std_dev_wire_harness_price = 0.87*2.20462*0.2  
mean_tarpaulin_waste_price = 0.38  # 380 USD/MT (US)
std_dev_tarpaulin_waste_price = 0.38*.2  
num_simulations = 10000  

# Bounds for the prices
lower_bound_wire_harness = 0  
upper_bound_wire_harness = 200
lower_bound_tarpaulin_waste = 0  
upper_bound_tarpaulin_waste = 100  

# Function to generate prices within bounds using a truncated normal distribution
def generate_truncated_normal(mean, std_dev, lower_bound, upper_bound, num_samples):
    a, b = (lower_bound - mean) / std_dev, (upper_bound - mean) / std_dev
    return truncnorm.rvs(a, b, loc=mean, scale=std_dev, size=num_samples)

# Generate feedstock prices ensuring they are within bounds
wire_harness_prices = generate_truncated_normal(mean_wire_harness_price, 
                                                std_dev_wire_harness_price, 
                                                lower_bound_wire_harness, 
                                                upper_bound_wire_harness, 
                                                num_simulations)

tarpaulin_waste_prices = generate_truncated_normal(mean_tarpaulin_waste_price, 
                                                   std_dev_tarpaulin_waste_price, 
                                                   lower_bound_tarpaulin_waste, 
                                                   upper_bound_tarpaulin_waste, 
                                                   num_simulations)

# Function to perform Goal Seek in Excel using COM interface
def goal_seek(ws_economic_analysis, target_cell, changing_cell, target_value):
    target_range = ws_economic_analysis.range(target_cell)
    changing_range = ws_economic_analysis.range(changing_cell)
    target_range.api.GoalSeek(Goal=target_value, ChangingCell=changing_range.api)
    return changing_range.value

# Store results
results10000 = []

# Initialize Excel application
app = xw.App(visible=False)
wb = xw.Book(file_path)

# Access relevant sheets
ws_opex = wb.sheets['OpEXParameters']
ws_economic_analysis = wb.sheets['Economic Analysis']

for i in range(num_simulations):
    # Update the feedstock prices in the relevant cells
    ws_opex.range('B3').value = wire_harness_prices[i]
    ws_opex.range('B4').value = tarpaulin_waste_prices[i]
    
    # Perform Goal Seek to calculate MSP
    msp = goal_seek(ws_economic_analysis, 'L55', 'U19', 0)  # Adjust cell references as needed
    results10000.append({
        'Wire Harness Price': wire_harness_prices[i],
        'Tarpaulin Waste Price': tarpaulin_waste_prices[i],
        'MSP': msp
    })

# Save results to a new Excel sheet or CSV file
results_df10000 = pd.DataFrame(results10000)
results_df10000.to_excel('.xlsx', index=False)

wb.save()
wb.close()
app.quit()

timefinal = time.time()

elapsedtime = timefinal-timeinitial 

print('The time it took was {} minutes.'.format(elapsedtime/60))
print(results_df10000.describe(include = 'all'))
